# Análise de Sentimentos de Tweets coletados em tempo real com Spark Streaming e MLlib

## Sumário

## Introdução

## 1) Definição do problema
### 1.1) O que é análise de sentimentos?
Uma tendência relativamente mais recente na análise de textos vai além da detecção de tópicos e tenta identificar a emoção por trás de um texto. Isso é chamado de análise de sentimentos, ou também de mineração de opinião e IA de emoção.

A análise de sentimentos é uma mineração contextual de um texto que identifica e extrai informações subjetivas no material de origem. Ela ajuda as empresas a entenderem o sentimento social de sua marca, produto ou serviço.

Um sistema de análise de sentimentos para conteúdo textual combina o processamento de linguagem natural (PLN) e técnicas de aprendizado de máquina para atribuir pontuações ponderadas de sentimento às sentenças. Com os recentes avanços na aprendizagem de máquina, o uso de técnicas avançadas de inteligência artificial se tornou eficaz para identificar sentimentos de usuários na web.

Quando uma empresa deseja entender o que estão falando sobre ela e qual a reputação de seus produtos online, uma das formas de se fazer isso é utilizando machine learning. Nesse sentido, uma das técnicas recomendadas é a análise de sentimentos, que consiste em extrair informações de textos a partir de linguagem natural.O objetivo dessa técnica é classificar sentenças, ou um conjunto de sentenças, como positivas ou negativas. Essa classificação é realizada automaticamente e extrai informações subjetivas de textos, criando conhecimento estruturado que pode ser utilizado por um sistema.

Contudo, obter uma grande quantidade de dados em tempo real pode gerar um problema. A dificuldade em termos de capacidade computacional para processar todos estes dados em tempo real. Ao longo dos últimos anos, a principal ferramenta utilizada para processar grandes quantidade de dados em tempo real é o Apache Spark. Em linhas gerais, o Apache Spark é uma estrtutura de código aberto desenvolvida para ser um mecanismo de processamento analítico para aplicações de processamento de dados distribuídos em larga escala e aprendizado de máquina em tempo real, ou seja, para grandes volumes de dados, o chamado Big Data.

### 1.2) Objetivos deste projeto
Este projeto tem por objetivo, utilizar a API do Twitter para obter tweets em tempo real de um determinado assunto, aplicar técnicas de MapReduce, transformação de dados com o framework Spark e utilizar o MLlib para classificar tweets como sentimento positivo ou negativo.

### 1.3) Metodologia
O projeto foi dividido em duas principais etapas. A primeira parte do projeto compreende a obtenção de um dataset rotulado contendo sentenças e suas respectivas classificações: 1 ou 0. Sendo 1 para sentimento positivo e 0 para sentimento negativo. O dataset foi obtido do repositório de datasets Kaggle e se trata de um conjunto de dados fornecido por um colaborador da Universidade de Michigan. O dataset se chama <a target="_blank" href="https://www.kaggle.com/datasets/seesea0203/umich-si650-nlp">UMICH SI650 NLP</a>, contém 5662 registros e duas colunas. A primeira para o texto e a segunda para o sentimento.

Após o carregamento dos dados foi realizado o pré-processamento dos dados. Esta é a principal atividade em um projeto baseado em dados. Os dados aqui se tratam de tweets reais e há vários tratamentos que devem ser realizados para garantir a legibilidade dos dados. Foi aplicado a remoção de caracteres especiais, remoção e de pontuação, remoção de stopwords (palavras irrelevantes para o sentido da frase) e a divisão dos dados em listas de palavras para se adequar ao formato de entrada que o modelo espera.

Com os dados limpos e organizados os mesmos foram submetidos ao algoritmo de Machine Learning responsável por fazer a classicação dos tweets e gerar o modelo de classificação. 

Em seguida foi iniciado a segunda parte do projeto, a etapa de coleta dos dados em tempo real do Twitter, tratamento dos tweets e a classificação com o modelo gerado anteriormente. 

A primeira atividade para obter dados do Twitter via API é estabelecar conexão com as respecitivas autenticações: api key, api key secret, access token, access token secret e bearer token. Com as chaves de autenticação foi necessário estabelecer conexão, obter as regras de acesso, deletar as antigas regras de acesso, adicionar as novas regras de acesso, para então realizar a coleta de dados.

Depois de pronto a conexão com a API do Twitter, os dados vêm são coletados por meio do streaming de dados do Spark (SparkStreaming) e salvos em objetos DSTreams. Os dados são então pré-processados e submetidos ao modelo para realizar a classificação. A coleta de dados termina após o tempo estabelicido pelo usuário ou então após a interrupção da conexão.

---

In [ ]:
#import pyspark.sql.functions as F
from pyspark.sql import Row #Converte RDDs em objetos do tipo Row
from pyspark.sql.functions import col, isnan, when, count # Encontra a contagem para valores None, Null, Nan, etc.
from pyspark.sql.types import IntegerType, FloatType

from pyspark.ml.feature import StringIndexer, OneHotEncoder #Converte strings em valores numéricos
from pyspark.ml.linalg import Vectors #Serve para criar um vetor denso
from pyspark.ml.evaluation import MulticlassClassificationEvaluator # Para avaliar o modelo com as métricas de avaliação.
from pyspark.ml.feature import RobustScaler, StandardScaler, MinMaxScaler, Normalizer # Métodos para escalas dos dados
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, GBTClassifier, LinearSVC # Algoritmos de ML
from pyspark.ml import Pipeline # Criação de um Pipeline de execução.
from pyspark.ml.functions import vector_to_array
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator # GridSearch e Validação Cruzada
from pyspark.ml.evaluation import BinaryClassificationEvaluator # Evaluator para classificação binária

from math import floor

In [ ]:
# Módulos usados
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json
#import re

In [ ]:
# Pacote NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [ ]:
# Frequência de update
INTERVALO_BATCH = 10

In [ ]:
# Criando o StreamingContext
ssc = StreamingContext(sc, INTERVALO_BATCH)

## Treinando o Classificador de Análise de Sentimento

Uma parte essencial da criação de um algoritmo de análise de sentimento (ou qualquer algoritmo de mineração de dados) é ter um conjunto de dados abrangente ou "Corpus" para o aprendizado, bem como um conjunto de dados de teste para garantir que a precisão do seu algoritmo atende aos padrões que você espera. Isso também permitirá que você ajuste o seu algoritmo a fim de deduzir melhores (ou mais precisas) características de linguagem natural que você poderia extrair do texto e que vão contribuir para a classificação de sentimento, em vez de usar uma abordagem genérica. Tomaremos como base o dataset de treino fornecido pela Universidade de Michigan, para competições do Kaggle (https://inclass.kaggle.com/c/si650winter11).


Esse dataset contém 1,578,627 tweets classificados e cada linha é marcada como: 

#### 1 para o sentimento positivo 
#### 0 para o sentimento negativo 

### Obtendo dados de treino

In [ ]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder.master("local").appName("tw-session").config("spark.some.config.option", "session").getOrCreate()

In [ ]:
# Lendo o arquivo texto e criando um RDD em memória com Spark
rdd = sc.textFile("data/train.csv")

In [ ]:
header = rdd.first()
rdd_body = rdd.filter(lambda x: header not in x)#.map(lambda l: l.split(','))

list_columns = header.replace('.', '_').upper().split(',')
list_columns

In [ ]:
rdd_body = rdd_body.map(lambda x: x.replace(',0', ';0')).map(lambda x: x.replace(',1', ';1'))

In [ ]:
rdd_body.take(10)

### Pré Processamento dos dados

In [ ]:
# Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação.
def get_row(line):
    row = line.split(';')
    
    tweet = row[0].strip()
    sentimento = int(re.sub('[^\d]+', '', row[1]))
    
    translator = str.maketrans({key: None for key in string.punctuation})
    #translator = re.compile('[%s]' % re.escape(string.punctuation))
    #tweet = regex.sub('', tweet)
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
        tweet_lower.append(word.lower())
    return (tweet_lower, sentimento)

In [ ]:
# Aplcia a função a cada linha do dataset
dataset_treino = rdd_body.map(lambda line: get_row(line))

In [ ]:
dataset_treino.take(2)

### Modelagem

In [ ]:
# Cria um objeto SentimentAnalyzer 
sentiment_analyzer = SentimentAnalyzer()

In [ ]:
# Obtém a lista de stopwords em Inglês 
stopwords_all = []
for word in stopwords.words('english'):
    stopwords_all.append(word)
    stopwords_all.append(word + '_NEG')

In [ ]:
# Obtém 10.000 tweets do dataset de treino e retorna todas as palavras que não são stopwords
dataset_treino_amostra = dataset_treino.take(1000)

In [ ]:
all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [ ]:
# Cria um unigram e extrai as features
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops, top_n = 200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

In [ ]:
# Treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)

In [ ]:
# Testa o classificador em algumas sentenças
test_sentence1 = [(['this', 'program', 'is', 'suck'], '')]
test_sentence2 = [(['tough', 'day', 'at', 'work', 'today'], '')]
test_sentence3 = [(['good', 'wonderful', 'amazing', 'awesome'], '')]
test_set1 = sentiment_analyzer.apply_features(test_sentence1)
test_set2 = sentiment_analyzer.apply_features(test_sentence2)
test_set3 = sentiment_analyzer.apply_features(test_sentence3)

In [ ]:
test_set1[0][0]

In [ ]:
classifier.classify(test_set1[0][0])

---

---

## Coleta de dados do Twitter para Classificação

In [ ]:
import requests
import configparser
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import json
import re
import time

### Obtendo chaves de autentifação e Definindo regras de pesquisa

In [ ]:
# Autenticação do Twitter 
api_key = 'Olk6wAb5NqOnXztjIs9bR6FaF'
api_key_secret = 'vkUwO1OGEjKH5CGlmLsKP1KBTYoESycOGopwAjVTlDqG98Ajzj'
access_token = '1228500338912743425-fLQRIKtrHJe9mkvWM7EXToYmtfzzxc'
access_token_secret = 'q1cx5rGXqmIqeWX6gBsMGG0ymGzYLTfAe4Eagxin9ohRh'
bearer_token = r'AAAAAAAAAAAAAAAAAAAAAHY5hgEAAAAA2FSk9Qi3r1OKDdlXRzhkzox9InI%3DR3U3eL0OI5G8ka9GO1OlXtrZnvfuktbrEJozzVwPz1LssHYWcG'

In [ ]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r

In [ ]:
def get_set_rules():
    print('Getting rules...')
    response = requests.get("https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth)
    print(json.dumps(response.json()))
    rules = response.json()
    print(f'Status {response.status_code}')
    
    #########################################################################
    print('\nDeleting all rules...')
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post("https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth, json=payload)
    
    print(json.dumps(response.json()))
    print(f'Status {response.status_code}')
    
    #########################################################################
    print('\nSetting rules...')
    sample_rules = [{'value': 'russia'},{'value': 'war'},{'value': 'putin'}]

    payload = {"add": sample_rules}
    response = requests.post("https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth, json=payload)
    print(json.dumps(response.json()))
    print(f'Status {response.status_code}')

In [ ]:
# Gainaing access and connecting to Twitter API using Credentials
client = tweepy.Client(bearer_token, api_key, api_key_secret, access_token, access_token_secret)

auth = tweepy.OAuth1UserHandler(api_key, api_key_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

get_set_rules()

### Pré processamento dos Dados

In [ ]:
stream = ssc.queueStream([], default = rdd)
type(stream)

In [ ]:
# Total de tweets por update
NUM_TWEETS = 1  

In [ ]:
# Essa função conecta ao Twitter e retorna um número específico de Tweets (NUM_TWEETS)
def tfunc(t, rdd):
    return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
    response = requests.get("https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True)
    print('Status code {}'.format(response.status_code))
    count = 0
    for line in response.iter_lines():
        try:
            if count > NUM_TWEETS:
                break
            post = json.loads(line.decode('utf-8'))
            contents = [post['data']['text']]
            count += 1
            yield str(contents)
        except:
            result = False

In [ ]:
stream = stream.transform(tfunc)
type(stream)

In [ ]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))
type(coord_stream)

In [ ]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
    sentence = [(tweet, '')]
    test_set = sentiment_analyzer.apply_features(sentence)
    print(tweet, classifier.classify(test_set[0][0]))
    return(tweet, classifier.classify(test_set[0][0]))

In [ ]:
# Essa função retorna o texto do Twitter
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        translator = str.maketrans({key: None for key in string.punctuation})
        tweet = tweet.translate(translator)
        tweet = tweet.split(' ')
        tweet_lower = []
        for word in tweet:
            tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

In [ ]:
# Cria uma lista vazia para os resultados
resultados = []

In [ ]:
# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
    counts = pairs.reduceByKey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)
    print(result)

In [ ]:
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

### Coleta e classificação

In [ ]:
# Start streaming
ssc.start() # Start the computation
ssc.awaitTerminationOrTimeout(120) # Wait for the computation to terminate
ssc.stop()

In [ ]:
cont = True
while cont:
    if len(resultados) > 5:
        cont = False

# Fim